In [ ]:
!pip install gtfs-kit #install gtfs-kit which will be used to mine the gtfs (transit data)
!pip install osmnx
import pandas as pd
import gtfs_kit as gk
import numpy as np
from scipy.spatial import KDTree
from sklearn.neighbors import BallTree
import numpy as np
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import plotly.express as px
import datetime as dt
from sklearn.metrics.pairwise import haversine_distances
import ast
import re

In [208]:
#get nodes and edges
path_edges = "https://github.com/deepikarao281199/CSCI6364_MIND_THE_GAP/raw/refs/heads/main/saopaulo_edges.csv"
edges_df = pd.read_csv(path_edges)#load transit graph - edges
path_nodes = "https://github.com/deepikarao281199/CSCI6364_MIND_THE_GAP/raw/refs/heads/main/saopaulo_nodes.csv"
nodes_df = pd.read_csv(path_nodes)#load transit graph - nodes

In [ ]:
# load the edges and nodes - then create the graph
edges_df['2'] = edges_df['2'].apply(lambda x: ast.literal_eval(re.sub(r"np\.float64\(([-0-9.eE]+)\)", r"\1", x)))#convert 'dict' to just dict - np.float objects caused error so we replace it
nodes_df['1'] = nodes_df['1'].apply(lambda x: ast.literal_eval(re.sub(r"np\.float64\(([-0-9.eE]+)\)", r"\1", x)))#convert 'dict' to just dict
edges_back = list(edges_df.itertuples(index = False, name = None))
nodes_back =  list(nodes_df.itertuples(index = False, name = None))
G = nx.MultiDiGraph()#create empty graph
G.add_nodes_from(nodes_back)
G.add_edges_from(edges_back)
print(G)

MultiDiGraph with 22056 nodes and 102362 edges


In [ ]:
correct_nodes, correct_edges = 22056, 102362
if (len(G.nodes) == correct_nodes) and len(G.edges) == correct_edges:
    print("Correct Graph generated!")
else:
    print("Incorrect graph generated, redownload edges and nodes, try again!")

Correct Graph generated!


The transit network is build. The next step is to find the average transit time from each neightborhood. Metrics we will get:
- number of walkable stops from each neighbourhood centroid
- num of schools/hospitals/clincs/pharmacies/grocery etc reachable within 30 mins
- mean travel time to key services eg. average time to nearest groceries
- average transfer count to key services
- average distance to key services
- num of connected subgraphs - disconnected areas = accessibility gaps

In [ ]:
#using open street map to get walkable paths
places = ['São Paulo, Brazil']
G_walk = ox.graph_from_place(places, network_type="walk")
G_walk

In [209]:
#load nearest osmx nearest node to each stop - from ML Final Project GTFS
path_gwalk = "https://github.com/deepikarao281199/CSCI6364_MIND_THE_GAP/raw/refs/heads/main/saopaulo_stops_with_osmx.csv"
stops_gwalk = pd.read_csv(path_gwalk).set_index('stop_id')
stops_gwalk.head()

,stop_lat,stop_lon,gwalk_nearest
stop_id,,,
18848,-23.554022,-46.671108,11287097095
18849,-23.546498,-46.691141,165473564
18850,-23.558094,-46.660205,3511236618
18851,-23.635039,-46.641239,2396880345
18852,-23.646033,-46.641028,29777031


In [ ]:
#pre-computing walkable edges
osm_walkable = nx.all_pairs_dijkstra_path_length(G_walk, cutoff = 400, weight = 'length')
osm_walkable_edges = {}

for source, dist_dict in osm_walkable:
    for target, dist in dist_dict.items():
        osm_walkable_edges[(source, target)] = dist

# print(osm_walkable_edges)

In [133]:
#convert to radians to accomodate distance metrics
stop_coords = np.radians(stops_gwalk[['stop_lat', 'stop_lon']]).values
#Ball tree creation using Haversine distance
stops_tree = BallTree(stop_coords, metric = 'haversine')

#a quarter mile or 400 meters is generally considered walkable
earth_r = 6371000  #earths radius
distance = 400 /earth_r #radius for 400 meters

walk_speed = 1.375 #average meters per second - https://en.wikipedia.org/wiki/Preferred_walking_speed
walk_max = 10 #maximum time to walk by transfer in mins

In [176]:
def get_closest_stop_nh(row):
    lat = row['CENTLAT']
    lon = row['CENTLONG']
    point = [[lat, lon]] #must be 2d
    nearest = stops_tree.query_radius(point, r = distance) #returns indices of neighbors within distance - stop_id is the indicies
    return nearest[0]

def get_closest_stop_service(row):
    lat = np.radians(row['latitude'])
    lon = np.radians(row['longitude'])
    point = [[lat, lon]] #must be 2d
    nearest = stops_tree.query_radius(point, r = distance) #returns indices of neighbors within distance - stop_id is the indicies
    return nearest[0]

In [ ]:
#pre-compute nearest stops from neighbourhood - store in list
nh_path = ""
nh_df = pd.read_csv(nh_path)
nh_withGWalk = nh_df.copy()
# nh_withGWalk['gwalk_nearest'] = ox.distance.nearest_nodes(G_walk, nh_df['CENTLON'], nh_df['CENTLAT'])#returns nearest osmx node for each neighbourhood
nh_withGWalk['nearest_stop_id'] = nh_withGWalk[['CENTLAT', 'CENTLON']].apply(get_closest_stop_nh, axis = 1)#returns nearest stop_id for each node

,Unnamed: 0,stop_id,stop_lat,stop_lon,gwalk_nearest
0,0,18848,-23.554022,-46.671108,11287097095
1,1,18849,-23.546498,-46.691141,165473564
2,2,18850,-23.558094,-46.660205,3511236618
3,3,18851,-23.635039,-46.641239,2396880345
4,4,18852,-23.646033,-46.641028,29777031
...,...,...,...,...,...
22051,22051,960011507,-23.595972,-46.741941,1629035497
22052,22052,960011508,-23.596402,-46.743436,1609619877
22053,22053,960011509,-23.604681,-46.747021,8932612436
22054,22054,960011510,-23.604420,-46.748357,1963366761


In [181]:
#pre-compute nearest stops from service(healthcare) - stored in list
hos_path = "https://github.com/deepikarao281199/CSCI6364_MIND_THE_GAP/raw/refs/heads/main/sao_paulo_healthcare_facilities.csv"
hos_df = pd.read_csv(hos_path)
hos_withGWalk = hos_df.copy()
#hos_withGWalk['gwalk_nearest'] = ox.distance.nearest_nodes(G_walk, hos_df['latitude'], hos_df['longitude'])#returns nearest osmx node for each service
hos_withGWalk['nearest_stop_id'] = hos_withGWalk[['latitude', 'longitude']].apply(get_closest_stop_service, axis = 1)#returns nearest stop_id for each node
hos_gwalk_nodes = dict(zip(hos_withGWalk['osm_id'], hos_withGWalk['nearest_stop_id']))

In [182]:
#pre-compute nearest stops from service(groceries) - stored in list
groc_path = "https://github.com/deepikarao281199/CSCI6364_MIND_THE_GAP/raw/refs/heads/main/sao_paulo_groceries.csv"
groc_df = pd.read_csv(groc_path)
groc_withGWalk = groc_df.copy()
#groc_withGWalk['gwalk_nearest'] = ox.distance.nearest_nodes(G_walk, groc_df['latitude'], groc_df['longitude'])#returns nearest osmx node for each service
groc_withGWalk['nearest_stop_id'] = groc_withGWalk[['latitude', 'longitude']].apply(get_closest_stop_service, axis = 1)#returns nearest stop_id for each node
groc_gwalk_nodes = dict(zip(groc_withGWalk['osm_id'], groc_withGWalk['nearest_stop_id']))

In [180]:
#pre-compute nearest stops from service(groceries) - stored in list
school_path = "https://github.com/deepikarao281199/CSCI6364_MIND_THE_GAP/raw/refs/heads/main/sao_paulo_schools_k12.csv"
school_df = pd.read_csv(school_path)
school_withGWalk = school_df.copy()
#school_withGWalk['gwalk_nearest'] = ox.distance.nearest_nodes(G_walk, school_df['latitude'], school_df['longitude'])#returns nearest osmx node for each service
school_withGWalk['nearest_stop_id'] = school_withGWalk[['latitude', 'longitude']].apply(get_closest_stop_service, axis = 1)#returns nearest stop_id for each node
school_gwalk_nodes = dict(zip(school_withGWalk['osm_id'], school_withGWalk['nearest_stop_id']))

In [193]:
#pre-compute Dijkstra for each start and end node - neighbourhood stop to service stops
nh_nodes = list(set([int(node) for nearest in nh_withGWalk['nearest_stop_id'] for node in nearest]))

school_nodes = [int(node) for nearest in school_withGWalk['nearest_stop_id'] for node in nearest]
hos_nodes = [int(node) for nearest in hos_withGWalk['nearest_stop_id'] for node in nearest]
groc_nodes = [int(node) for nearest in groc_withGWalk['nearest_stop_id'] for node in nearest]

service_nodes = school_nodes + hos_nodes + groc_nodes
service_nodes = list(set(service_nodes))
print(len(service_nodes))

dist_dict = {}
time_dict = {}

for n in nh_nodes:
    for s in service_nodes:
        dist_dict[(n, s)] = nx.shortest_path_length(G, source=n, target=s, weight='distance')
        dist_dict[(n, s)] = nx.shortest_path_length(G, source=n, target=s, weight='time')

11378


In [ ]:
#num of schools/hospitals/clincs/pharmacies/grocery etc reachable within 30 mins
def num_service_reachable(nh, service_type):
    #get the stops within walking distance from neighbourhood
    nh_stops = nh['nearest_stop_id']
    if service_type == 'hospital':
        service_dict = hos_withGWalk
    elif service_type == 'school':
        service_dict = school_withGWalk
    elif service_type == 'grocery':
        service_dict = groc_withGWalk

    reachable = 0
    for nh in nh_stops:
        #for each stop get the stops for each service and then use the pre - computed to get the time
        for serv_stops in service_dict.values():
            if len(serv_stops) != 0: #not empty
                for stop in serv_stops:
                    service_time = time_dict[(nh, stop)]
                    if service_time <= 30:#the time is in mins so anything > 30 we don't add to reachable
                        reachable+=1

    return reachable

In [ ]:
#mean travel time to key services eg. average time to nearest groceries
def avg_service_reachable(neighbourhood):
    centroid_lat = neighbourhood['CENTLAT']
    centroid_lONG = neighbourhood['CENTLAT']
    reachable = []
    #get the stops within walking distance from neighbourhood
    #get the stops within walking distance from service
    #use dijkstra to get shortest paths between each neighbourhood stop and service stop
    #append time taken to reachable
    #return average of reachle
